<a href="https://colab.research.google.com/github/Kalidsa/ERAMIA-2025-carros-eletricos/blob/main/ERAMIA_2025_coleta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install asyncpraw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 4.7 MB/s eta 0:00:00


In [ ]:
import asyncpraw
import pandas as pd
import re
import datetime
import asyncio

In [ ]:
from huggingface_hub import login
from google.colab import userdata

login(token=userdata.get('hugg'))


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: fineGrained).
The token `pessoal` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate w

In [ ]:
import re
from transformers import pipeline

sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment"
)

def tem_sentimento(texto):
    texto = texto.strip()

    if len(texto) < 15 or texto.lower().startswith(("http", "edit:", "1.")):
        return False
    try:
        resultado = sentiment_pipeline(texto[:512])[0]
        label = resultado['label'].lower()
        score = resultado['score']

        estrelas = int(label[0])
        if estrelas <= 2 and score >= 0.6:
            return True
        elif estrelas == 3 and score >= 0.75:
            return True
        elif estrelas >= 4 and score >= 0.6:
            return True
        else:
            return False
    except:
        return False



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
import psutil
import os
import time

process = psutil.Process(os.getpid())
start_time = time.time()

async def main():
    reddit = asyncpraw.Reddit(
        client_id=userdata.get('CLIENT_ID'),
        client_secret=userdata.get('CLIENT_SECRET'),
        user_agent=userdata.get('USER_AGENT'),
    )

    data_min = datetime.datetime(2015, 1, 1).timestamp()
    data_max = datetime.datetime(2025, 6, 3, 23, 59, 59).timestamp()

    termos_busca = [
    "carro", "elétrico", "carros", "elétricos", "veículos",
    "carro elétrico", "carros elétricos", "veículo elétrico", "veículos elétricos",
    "carro Tesla", "BYD", "Volvo elétrico", "Neta X", "Yuan Pro", "Dolphin", "Dolphin Mini",
    "Peugeot e-2008", "Volvo EX30", "GWM Ora 03", "Renault Kwid E-Tech",
    "JAC E-JS1", "Neta Aya"
]


    subreddit = await reddit.subreddit("brasil+carros+eletricos")
    comentarios_filtrados = []

    for termo in termos_busca:
        try:
            async for submission in subreddit.search(termo, sort="new", limit=500):
                await asyncio.sleep(1.5)
                if submission.created_utc < data_min:
                    continue

                await submission.load()
                await submission.comments.replace_more(limit=0)

                for comment in submission.comments.list():
                    if not (data_min <= comment.created_utc <= data_max):
                        continue

                    texto = comment.body.strip()
                    if (
                        30 < len(texto) < 1000 and
                        '\n' not in texto and
                        not texto.startswith("http") and
                        not texto.lower().startswith("edit:") and
                        not texto.lower().startswith("1.") and
                        any(t.lower() in texto.lower() for t in termos_busca) and
                        tem_sentimento(texto)
                    ):
                        comentarios_filtrados.append({
                            "id": comment.id,
                            "comentario": texto,
                            "termo": termo,
                            "data_comentario": datetime.datetime.fromtimestamp(comment.created_utc).strftime("%Y-%m-%d %H:%M:%S")
                        })
        except Exception as e:
            print(f"⚠️ Erro durante busca ou coleta: {e}")
            await asyncio.sleep(60)

    # Remover duplicatas
    vistos = set()
    comentarios_unicos = []
    for c in comentarios_filtrados:
        if c["id"] not in vistos:
            vistos.add(c["id"])
            comentarios_unicos.append(c)

    df = pd.DataFrame(comentarios_unicos)
    df.to_csv("comentarios_reddit.csv", index=False, encoding="utf-8")

    print(f"\n🔎 {len(df)} comentários únicos salvos em 'comentarios_reddit.csv'.")

    await reddit.close()

await main()


print("✅ Processo concluído com sucesso.")
end_time = time.time()
print(f"⏱ Tempo total: {end_time - start_time:.2f}s")
print(f"🧠 Memória usada: {process.memory_info().rss / 1024**2:.2f} MB")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



🔎 1039 comentários únicos salvos em 'comentarios_reddit_2015_ate_2025.csv'.
✅ Processo concluído com sucesso.
⏱ Tempo total: 5031.75s
🧠 Memória usada: 1848.14 MB


In [ ]:
!pip install google-api-python-client pandas nltk
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 51.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=64dbd1489ca0aaaabee7ead5e7e4d52c37397b53243ef2c058be5daef65e9a07
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [ ]:
!pip install tqdm


In [ ]:
termos_busca = [
    "carro", "elétrico", "carros", "elétricos", "veículos",
    "carro elétrico", "carros elétricos", "veículo elétrico", "veículos elétricos",
    "carro Tesla", "BYD", "Volvo elétrico", "Neta X", "Yuan Pro", "Dolphin", "Dolphin Mini",
    "Peugeot e-2008", "Volvo EX30", "GWM Ora 03", "Renault Kwid E-Tech",
    "JAC E-JS1", "Neta Aya"
]


VIDEO_IDS = [
    "PkkBOz2xM3s", "ZkqfKaEKw3Q", "PwHZ4fy5zl0","i3inJQV6TfA","ZQzNkF2SIso","_ZpDBoY_Qc8","rVVgTB--tWQ","E4kNKb4CaQM","Y9DHOOFcjl8","vg3oe6sfXsw",
    "tq8wjMR1QqQ","JUCVwYllxug","ju3Quli4gVc","PXyyXJjL9vI","ZKjtU9Wg6D8","ZQzNkF2SIso","5Td5M0zyeBE","PWinNdl-uSo","-j50dgnR8eA","D2WVMk8p9kA",
    "cwIDHqFEWwc","tuDLQ9clP5I","EWSXuxWSMWQ","XuRKYpNYoIY","KmrBRhvqJxw","UEO9VzgMEOE","stydxkp-pIU","JY49lv76Fm0","75iIA8yPgJo","7QRVD5AHgGU",
    "10uB33EqP9c","lD-g0pOBwtg","Mmv5Mqw4a4s","z5rVMn5J3pU","TQ0vCjU_cEU","FghV4R2jKeo","yxo_WtuH_5E","5RY_3tk0vZc","SnrkoBDtVUo","hKp9sEarQwU",
    "9ADPyCyI2iE", "7T_DCXuq5FM", "CSopJF6U9ZI", "KjDxVRNkiN8", "h_7ceo7H8AM", "CYIvimGnpPc", "0HFhP9ymscc", "9n-xhO0zQcU", "CMlnXx8ZoHA","Gk7oLqWZvZs",

]

SHORTS_IDS = [
    "fxJKfp_Oj20","TqRitNiXvdM", "ySzsMcqv0-g","3feuQVTNHbw","MNCbHEew5R8","wpx_rGK0G-c","sP7WJNHQ2Ec","TtD8dHB7mz4","Eyey-3Ywd8Q","jw2RWtj9_Pg",
    "p2HbY8IlRcI","x1OwilwEw1w","tfxVPW2kJLc","vjtpFySZ_Yo","uaEfjMKwaUc","YtqRZMT46lU","frquR7piyPE" , "aK3UvRK1SZs","-bUNuQ9B1fg","NnEjRx2a8YA",
    "pMqDUVRLzeE","FUs7sUyqX6Q","OPHliQvtlfg","z0gKgThZVxE","YqmKOsI9iMo","fdKwxob4zS8","E5DNsxpPXhc","xmRaOVZsjUc","ceoGncoBnus","sIgmo_5VI6g",
    "D-8vSZoDLmc","NzX5PDEGH_I","_SIl7ndMH9Y","GGf4j_H1UVM","VwlqfK8YoyI", "rry6z9_G2mM","BZOJSYldcyY", "c4o8T38WOsc","mGIGDaedxzg","anq5oKc_GFk",
    "FcVpqQdMOt4","bxLHdLX0ViM","wwPzcNk86t8","xB9q4lg9xl0","OEgy7ZxFn1o","Bniu_x0-P98","1bS2qoq9Cio","q_gaLN4gXyo","wp1qfezUx3Q","JtqOXYvcZjg",
    "u4MqciYE_2Q", "5NodtFnnMfY","xbfyEZrXja0","zjM2N_u2v7g","qMBUgzgTwIw","KSBP-DsgcYk","nG748nytgkg","s6VWJ9-jS1Y","Co32pPDpDl4","0c6TefDP8dg",
    "CHmBnHBXGk4","SiLfxT9i1vc","7oFLCJyHjAY","iVijrVASQow","JbO2QPlDwu0","xNKusFKchBQ","FUTF9xNF7z8","7azqCp3y39c","Cf1FCIy3vlA","w7s8kVrtL-o",
    "akidAghUBMA","BkZcUQSiqO4","lFgCy_eYZzg","M8ZCelUS4tw","oHOGvXyW5jc","skFka7O1q6g","YyQVHOKKelU","2I6KM6O6_D8","2oJc6S3PJFQ","TN_HFEIODB4",
    "Yx6AQJle0to","IN5C2-wl5kg","BYGtt2zkkaw","POChzK4M0c4","kqNvk-ulqiE","1AJjstYr8I8","Qbru5vFuuGI","EhrWZCKIHlE","pd55DBk0ZJY","veIqkTofW4k",
    "pnim6LX6rcE","W6nhokgnk-8","zw-pgpnksq4","VItF3LdUVy8","vFMEKbP2KK4","7KJlg9n_wDw","qGbw_w6IYmY","PQH6IsvZDoA","DAyAbRFSreQ","9ITLB6x3wLI"

]

def contem_termo_busca(texto):
    texto_lower = texto.lower()
    return any(termo.lower() in texto_lower for termo in termos_busca)

def comentarios_ativados(video_id):
    youtube = build('youtube', 'v3', developerKey=userdata.get('API_KEY'))
    try:
        req = youtube.videos().list(
            part="statistics",
            id=video_id
        )
        resp = req.execute()
        stats = resp["items"][0]["statistics"]
        return "commentCount" in stats and int(stats["commentCount"]) > 0
    except:
        return False

In [ ]:
import os
import time
import psutil
from tqdm import tqdm
from datetime import datetime
from langdetect import detect
from googleapiclient.errors import HttpError
from googleapiclient.discovery import build

process = psutil.Process(os.getpid())
start_time = time.time()

def coletar_comentarios(video_id, max_results=5000):
    youtube = build('youtube', 'v3', developerKey=userdata.get('API_KEY'))
    comentarios = []

    try:
        req = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=100,
            textFormat="plainText"
        )
        resp = req.execute()

        while resp and len(comentarios) < max_results:
            for item in resp["items"]:
                snippet = item["snippet"]["topLevelComment"]["snippet"]
                texto = snippet["textDisplay"]
                try:
                    if detect(texto) != "pt":
                        continue
                except:
                    continue

                if len(texto) > 30 and not texto.startswith("http"):
                    data_comentario = datetime.strptime(snippet["publishedAt"], "%Y-%m-%dT%H:%M:%SZ")
                    if datetime(2015, 1, 1) <= data_comentario <= datetime.now():
                        if contem_termo_busca(texto) and tem_sentimento(texto):
                            comentarios.append({
                                "id": item["id"],
                                "video_id": video_id,
                                "autor": snippet["authorDisplayName"],
                                "comentario": texto,
                                "data": snippet["publishedAt"],
                            })

            if "nextPageToken" in resp and len(comentarios) < max_results:
                time.sleep(1);
                req = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=100,
                    pageToken=resp["nextPageToken"],
                    textFormat="plainText"
                )
                resp = req.execute()
            else:
                break

    except HttpError as e:
        if e.resp.status == 403:
            print(f"⚠️ Comentários desativados para o vídeo {video_id}, pulando.")
        else:
            print(f"❌ Erro ao coletar comentários do vídeo {video_id}: {e}")

    return comentarios

# ---------- Execução principal ------------


todos_comentarios = []

for vid in tqdm(VIDEO_IDS, desc="📽️ Coletando comentários"):
    if comentarios_ativados(vid):
        comentarios = coletar_comentarios(vid, max_results=5000)
        todos_comentarios.extend(comentarios)
    else:
        print(f"⚠️ Comentários desativados ou indisponíveis para o vídeo {vid}, pulando.")



for vid in tqdm(SHORTS_IDS, desc="📽️ Coletando comentários de Shorts"):
    if comentarios_ativados(vid):
        comentarios = coletar_comentarios(vid, max_results=5000)
        todos_comentarios.extend(comentarios)
    else:
        print(f"⚠️ Comentários desativados ou indisponíveis para o vídeo {vid}, pulando.")

# Remover duplicatas por ID
df = pd.DataFrame(todos_comentarios).drop_duplicates(subset="id")
df.to_csv("/content/drive/MyDrive/SEPOC - 2025 - BC/carros_eletricos_review.csv", index=False, encoding="utf-8")

print(f"\n✅ {len(df)} comentários salvos em 'carros_eletricos_review.csv'")



print("✅ Processo concluído com sucesso.")
end_time = time.time()
print(f"⏱ Tempo total: {end_time - start_time:.2f}s")
print(f"🧠 Memória usada: {process.memory_info().rss / 1024**2:.2f} MB")

📽️ Coletando comentários de Shorts:  87%|████████▋ | 87/100 [19:03<06:20, 29.26s/it]

⚠️ Comentários desativados ou indisponíveis para o vídeo EhrWZCKIHlE, pulando.


📽️ Coletando comentários de Shorts: 100%|██████████| 100/100 [20:04<00:00, 12.04s/it]


✅ 8676 comentários salvos em 'carros_eletricos_review.csv'
✅ Processo concluído com sucesso.
⏱ Tempo total: 2228.04s
🧠 Memória usada: 2018.55 MB


In [ ]:
import pandas as pd

df_youtube = pd.read_csv("/content/drive/MyDrive/SEPOC - 2025 - BC/carros_eletricos_review.csv")
df_reddit = pd.read_csv("/content/drive/MyDrive/SEPOC - 2025 - BC/comentarios_reddit_2015_ate_2025.csv")

df_reddit = df_reddit.rename(columns={"data_comentario": "data"})
df_reddit = df_reddit.rename(columns={"comentario": "content"})
df_youtube = df_youtube.rename(columns={"comentario": "content"})

df_youtube = df_youtube[['content', 'data']].copy()
df_youtube['origem'] = 'youtube'

df_reddit = df_reddit[['content', 'data']].copy()
df_reddit['origem'] = 'reddit'


df_comentarios = pd.concat([df_youtube, df_reddit], ignore_index=True)

df_comentarios.dropna(subset=['content'], inplace=True)
df_comentarios = df_comentarios[df_comentarios['content'].str.strip() != '']

df_comentarios.to_csv('/content/drive/MyDrive/SEPOC - 2025 - BC/comentarios_reddit_youtube.csv', index=False)

print(df_comentarios.head())

                                             content                  data  \
0  pena que esses carros não sobem nem um morrinh...  2025-06-08T12:21:21Z   
1  Trás o elétrico que carrega em 5 minutos de 0 ...  2025-06-02T17:14:42Z   
2  se esses carros der falha e sismar de quequer ...  2025-05-31T00:50:35Z   
3                  Agora, faz a bola perfeita de BYD  2025-05-29T23:07:59Z   
4  A BYD é a rainha do marketing, viram que no Br...  2025-05-28T19:33:26Z   

    origem  
0  youtube  
1  youtube  
2  youtube  
3  youtube  
4  youtube  


In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px

origem_counts = df_comentarios['origem'].value_counts()

origem_counts_df = origem_counts.reset_index()
origem_counts_df.columns = ['origem', 'count']

fig1 = px.bar(
    origem_counts_df,
    x='origem',
    y='count',
    title='Número de Comentários por Plataforma',
    color='origem',
    text='count'
)
fig1.update_layout(yaxis_title='Quantidade', xaxis_title='Origem')
fig1.show()

In [ ]:
df_comentarios['data'] = pd.to_datetime(df_comentarios['data'], errors='coerce')

df_comentarios['ano'] = df_comentarios['data'].dt.year

comentarios_por_ano = df_comentarios['ano'].value_counts().sort_index()
comentarios_por_ano_df = comentarios_por_ano.reset_index()
comentarios_por_ano_df.columns = ['ano', 'count']

fig2 = px.bar(
    comentarios_por_ano.reset_index(),
    x='ano',
    y='count',
    labels={'index': 'Ano', 'ano': 'Quantidade'},
    title='Número de Comentários por Ano',
    color='count',
    text='count'
)
fig2.update_layout(
    yaxis_title='Quantidade',
    xaxis_title='Ano'
)
fig2.update_traces(textposition='outside')
fig2.show()